#   MLlib: La librería de aprendizaje automático escalable de Apache Spark

MLlib es la biblioteca de aprendizaje automático (Machine Learning) de Apache Spark. Está diseñada para ser escalable y eficiente, permitiendo realizar tareas de aprendizaje automático en grandes volúmenes de datos de manera distribuida.

## Características de MLlib:

### 1. Escalabilidad:

Puede manejar grandes volúmenes de datos distribuidos en clústeres, lo que la hace adecuada para big data.

### 2.Rendimiento:

Está optimizada para proporcionar un rendimiento rápido y eficiente, aprovechando el procesamiento en memoria de Spark.

### 3.Facilidad de uso:

Ofrece una API sencilla y consistente, disponible en múltiples lenguajes como Java, Scala, Python y R.

###  4.Interoperabilidad:

Puede integrarse fácilmente con otras bibliotecas de Spark, como Spark SQL y DataFrames, facilitando el preprocesamiento de datos y la manipulación de conjuntos de datos.

### 5.Amplitud de algoritmos:

Proporciona una amplia gama de algoritmos de aprendizaje automático, que incluyen clasificación, regresión, clustering, filtrado colaborativo y reducción de dimensionalidad.

## Componentes principales de MLlib:

### 1.Algoritmos de aprendizaje automático:

- Clasificación: Ejemplos incluyen la regresión logística y el árbol de decisión.
- Regresión: Ejemplos incluyen la regresión lineal y la regresión de árboles.
- Clustering: Ejemplo incluye k-means.
- Filtrado colaborativo: Ejemplo incluye la factorización de matrices para recomendaciones.

### 2.Pipelines:

- Permite crear flujos de trabajo de Machine Learning reutilizables y configurables, que incluyen etapas de preprocesamiento, transformación de datos y modelado.

### 3.Transformaciones y preprocesamiento de datos:

- Herramientas para normalización de datos, escalado, manejo de valores faltantes, codificación de variables categóricas, etc.

### 4.Evaluación de modelos:

- Métricas y herramientas para evaluar el rendimiento de los modelos, como precisión, recall, F1 score, entre otros.

### 5.Persistencia de modelos:

- Soporte para guardar y cargar modelos entrenados para reutilizarlos o para producción.

MLlib se destaca por su capacidad para manejar datos a gran escala y por su integración nativa con el ecosistema Spark, lo que facilita la creación de pipelines de datos y modelos de Machine Learning robustos y escalables


En este notebook aprenderemos a entrenar modelos de Machine Learning con Spark, utilizando la librería de MLlib

In [1]:
!pip install jupyter_contrib_nbextensions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 20.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.9/466.9 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 39.8 MB/s eta 0:00:00
  Created wheel for jupyter_contrib_nbextensions: filename=jupyter_contrib_nbextensions-0.7.0-py2.py3-none-any.whl size=23428780 sha256=72bc4faf19e0155b77b676d0f31961b684c63e578171d579a6ab1f45da2cd4c8
  Stored in directory: /root/.cache/pip/wheels/ea/cc/7d/99ef154f984726b1201c0f72cfe9c9d7c5132c1a2ae4d8677f
  Created wheel for jupyter_contrib_core: filename=jupyter_contrib_core-0.4.2-py2.py3-none-any.whl size=17482 sha256=c8083b42fda93b80d6806ddbf34e1b804cd8d6db6fb67bb6b5d1d7fad2f9bbf3
  Stored in directory: /root/.cache/pip/wheels/a9/52/88/e0643cdfd68f0562087918c37dd583378648dbc3df68b907f7
Successfully built jupyter_contrib_nbextensions jupyter_contrib_core


In [4]:
!pip install findspark pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=7ac04a127dd86944610cb14210fbbcaae49492bf81f7243bdf5b4cadc1ef5e8a
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


#### Fase 1. Importación y análisis de datos

Pimero, instalemos cualquier dependencia necesaria y configuremos nuestro entorno Spark.

In [5]:
import findspark
findspark.init()

import pandas as pd
import pyspark

In [6]:
from pyspark.sql import SparkSession


spark = SparkSession.builder \
    .appName("AnalisisNotasEstudiantes") \
    .getOrCreate()


In [7]:
spark

In [8]:
# Datos de personas
per = [
    (0, "Carlos", "Astudillo", "0000000001", "0958880331"),
    (1, "Jonnathan", "Astudillo", "0000000002", "0985662514"),
    (2, "Danny", "Auquilla", "0000000003", "0990000001"),
    (3, "Victor", "Cabrera", "0000000004", "0993118236"),
    (4, "Paula", "Gomez", "0000000005", "0998264130"),
    (5, "Maicol", "Lojano", "0000000006", "099486071"),
    (6, "Kevin", "Lojano", "0000000007", "0987799450"),
    (7, "Jose", "Muños", "0000000008", "0997026632"),
    (8, "Danny", "Pineda", "0000000009", "0963325761"),
    (9, "Paul", "Rodriguez", "0106422496", "0998938189"),
    (10, "Ariel", "Saquicela", "0000000010", "0978998281"),
    (11, "Calva", "Morocho", "0000000011", "0000000001")
]

# Datos de materias
mat = [
    ("0MD", "Mineria de datos", 7),
    ("1MR", "Marcos de referencia", 5),
    ("2IN", "Inteligencia de negocios", 5),
    ("3EP", "Etica Profesional", 2),
    ("4E", "Emprendimiento", 3),
    ("5AP", "Aprendizaje profundo", 5)
]

# Datos de cursos (id_curso, id_estudiante, id_materia, asistencia, nota_diagnostico, nota_parcial, nota_final)
curso = [
    ("M3A", 2, "1MR", 90, 6.0, 6.5, 5.91),
    ("M3A", 3, "1MR", 85, 7.0, 8.0, 8.79),
    ("M3A", 5, "1MR", 95, 9.0, 9.5, 9.32),
    ("M3A", 9, "1MR", 88, 8.0, 9.0, 9.13),
    ("M3A", 10, "1MR", 92, 6.0, 6.5, 6.17),
    ("M3A", 0, "1MR", 100, 9.5, 9.0, 9.40),
    ("M3A", 1, "0MD", 75, 7.0, 8.5, 9.17),
    ("M3A", 2, "0MD", 80, 8.0, 8.5, 8.53),
    ("M3A", 3, "0MD", 85, 8.5, 8.5, 8.63),
    ("M3A", 4, "0MD", 90, 9.0, 9.0, 9.24),
    ("M3A", 5, "0MD", 95, 9.0, 9.0, 9.05),
    ("M3A", 7, "0MD", 90, 8.5, 9.0, 9.71),
    ("M3A", 8, "0MD", 88, 8.0, 8.5, 8.74),
    ("M3A", 9, "0MD", 85, 8.0, 9.0, 9.28),
    ("M3A", 10, "0MD", 75, 6.0, 6.0, 5.78),
    ("M3A", 2, "4E", 80, 8.5, 9.0, 9.77),
    ("M3A", 3, "4E", 85, 8.5, 9.0, 9.11),
    ("M3A", 11, "4E", 90, 6.0, 7.0, 6.59),
    ("M3A", 6, "4E", 75, 4.0, 5.0, 4.59),
    ("M3A", 9, "4E", 92, 8.5, 9.0, 9.17),
    ("M3A", 10, "4E", 85, 6.0, 6.5, 6.08),
    ("M3A", 0, "5AP", 90, 8.0, 8.5, 8.74),
    ("M3A", 1, "5AP", 85, 7.0, 7.5, 7.64),
    ("M3A", 2, "5AP", 80, 5.5, 5.0, 5.63),
    ("M3A", 3, "5AP", 85, 8.0, 8.5, 8.68),
    ("M3A", 4, "5AP", 90, 8.5, 8.0, 8.57),
    ("M3A", 5, "5AP", 95, 9.0, 9.5, 9.06),
    ("M3A", 6, "5AP", 75, 2.0, 3.0, 2.75),
    ("M3A", 7, "5AP", 88, 8.5, 9.0, 9.01),
    ("M3A", 8, "5AP", 85, 8.0, 8.5, 8.39),
    ("M3A", 9, "5AP", 90, 8.5, 9.0, 9.15),
    ("M3A", 10, "5AP", 85, 8.0, 8.0, 8.35),
    ("M3A", 0, "2IN", 70, 1.0, 1.0, 1.00),
    ("M3A", 1, "2IN", 75, 1.0, 1.0, 1.00),
    ("M3A", 2, "2IN", 80, 1.0, 1.0, 1.00),
    ("M3A", 3, "2IN", 85, 1.0, 1.0, 1.00),
    ("M3A", 5, "2IN", 90, 1.0, 1.0, 1.00),
    ("M3A", 6, "2IN", 95, 1.0, 1.0, 1.00),
    ("M3A", 8, "2IN", 80, 1.0, 1.0, 1.00),
    ("M3A", 10, "2IN", 85, 1.0, 1.0, 1.00),
    ("M3A", 3, "2IN", 90, 1.0, 1.0, 1.00),
    ("M3A", 11, "2IN", 95, 1.0, 1.0, 1.00),
    ("M3A", 6, "2IN", 70, 1.0, 1.0, 1.00),
    ("M3A", 10, "2IN", 75, 1.0, 1.0, 1.00)
]

# Crear DataFrames
per_df = spark.createDataFrame(per, ["id_persona", "nombre", "apellido", "cedula", "celular"])
mat_df = spark.createDataFrame(mat, ["id_materia", "nombre_materia", "nro_horas"])
curso_df = spark.createDataFrame(curso, ["id_curso", "id_estudiante", "id_materia", "asistencia", "nota_diagnostico", "nota_parcial", "nota_final"])

# Mostrar DataFrames
per_df.show()
mat_df.show()
curso_df.show()










+----------+---------+---------+----------+----------+
|id_persona|   nombre| apellido|    cedula|   celular|
+----------+---------+---------+----------+----------+
|         0|   Carlos|Astudillo|0000000001|0958880331|
|         1|Jonnathan|Astudillo|0000000002|0985662514|
|         2|    Danny| Auquilla|0000000003|0990000001|
|         3|   Victor|  Cabrera|0000000004|0993118236|
|         4|    Paula|    Gomez|0000000005|0998264130|
|         5|   Maicol|   Lojano|0000000006| 099486071|
|         6|    Kevin|   Lojano|0000000007|0987799450|
|         7|     Jose|    Muños|0000000008|0997026632|
|         8|    Danny|   Pineda|0000000009|0963325761|
|         9|     Paul|Rodriguez|0106422496|0998938189|
|        10|    Ariel|Saquicela|0000000010|0978998281|
|        11|    Calva|  Morocho|0000000011|0000000001|
+----------+---------+---------+----------+----------+

+----------+--------------------+---------+
|id_materia|      nombre_materia|nro_horas|
+----------+-------------------

In [9]:
curso_df.show(5)

+--------+-------------+----------+----------+----------------+------------+----------+
|id_curso|id_estudiante|id_materia|asistencia|nota_diagnostico|nota_parcial|nota_final|
+--------+-------------+----------+----------+----------------+------------+----------+
|     M3A|            2|       1MR|        90|             6.0|         6.5|      5.91|
|     M3A|            3|       1MR|        85|             7.0|         8.0|      8.79|
|     M3A|            5|       1MR|        95|             9.0|         9.5|      9.32|
|     M3A|            9|       1MR|        88|             8.0|         9.0|      9.13|
|     M3A|           10|       1MR|        92|             6.0|         6.5|      6.17|
+--------+-------------+----------+----------+----------------+------------+----------+
only showing top 5 rows



In [11]:
curso_df.toPandas().head()

,id_curso,id_estudiante,id_materia,asistencia,nota_diagnostico,nota_parcial,nota_final
0,M3A,2,1MR,90,6.0,6.5,5.91
1,M3A,3,1MR,85,7.0,8.0,8.79
2,M3A,5,1MR,95,9.0,9.5,9.32
3,M3A,9,1MR,88,8.0,9.0,9.13
4,M3A,10,1MR,92,6.0,6.5,6.17


In [12]:
curso_df.show(5)

+--------+-------------+----------+----------+----------------+------------+----------+
|id_curso|id_estudiante|id_materia|asistencia|nota_diagnostico|nota_parcial|nota_final|
+--------+-------------+----------+----------+----------------+------------+----------+
|     M3A|            2|       1MR|        90|             6.0|         6.5|      5.91|
|     M3A|            3|       1MR|        85|             7.0|         8.0|      8.79|
|     M3A|            5|       1MR|        95|             9.0|         9.5|      9.32|
|     M3A|            9|       1MR|        88|             8.0|         9.0|      9.13|
|     M3A|           10|       1MR|        92|             6.0|         6.5|      6.17|
+--------+-------------+----------+----------+----------------+------------+----------+
only showing top 5 rows



In [13]:
curso_df.count()

44

In [14]:
curso_df.dtypes

[('id_curso', 'string'),
 ('id_estudiante', 'bigint'),
 ('id_materia', 'string'),
 ('asistencia', 'bigint'),
 ('nota_diagnostico', 'double'),
 ('nota_parcial', 'double'),
 ('nota_final', 'double')]

In [15]:
from pyspark.sql.types import *

# Crear sesión de Spark
spark = SparkSession.builder \
    .appName("AnalisisNotasEstudiantes") \
    .getOrCreate()

# Definir esquemas para los DataFrames

# Esquema para el DataFrame de personas
personas_schema = StructType([
    StructField('id_persona', IntegerType(), True),
    StructField('nombre', StringType(), True),
    StructField('apellido', StringType(), True),
    StructField('cedula', StringType(), True),
    StructField('celular', StringType(), True)
])

# Esquema para el DataFrame de materias
materias_schema = StructType([
    StructField('id_materia', StringType(), True),
    StructField('nombre_materia', StringType(), True),
    StructField('nro_horas', IntegerType(), True)
])

# Esquema para el DataFrame de cursos
cursos_schema = StructType([
    StructField('id_curso', StringType(), True),
    StructField('id_estudiante', IntegerType(), True),
    StructField('id_materia', StringType(), True),
    StructField('asistencia', IntegerType(), True),
    StructField('nota_diagnostico', DoubleType(), True),
    StructField('nota_parcial', DoubleType(), True),
    StructField('nota_final', DoubleType(), True)
])

In [16]:
curso_df.printSchema()

root
 |-- id_curso: string (nullable = true)
 |-- id_estudiante: long (nullable = true)
 |-- id_materia: string (nullable = true)
 |-- asistencia: long (nullable = true)
 |-- nota_diagnostico: double (nullable = true)
 |-- nota_parcial: double (nullable = true)
 |-- nota_final: double (nullable = true)



In [17]:
# Basics stats from our columns
curso_df.describe().toPandas()

,summary,id_curso,id_estudiante,id_materia,asistencia,nota_diagnostico,nota_parcial,nota_final
0,count,44,44,44,44,44,44,44
1,mean,None,5.431818181818182,None,85.4090909090909,5.7272727272727275,6.056818181818182,6.139318181818182
2,stddev,None,3.446587306190954,None,7.221795762693306,3.2358000079318754,3.389820447139905,3.494623377070655
3,min,M3A,0,0MD,70,1.0,1.0,1.0
4,max,M3A,11,5AP,100,9.5,9.5,9.77


In [20]:
!pip install pandas-profiling


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 359.5/359.5 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.8/104.8 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 686.1/686.1 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 15.3 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27080 sha256=9be56fbee4b8cee1695a40584e3fe25603bf2faa207914b9667a4d671ddba382
  Stored in directory: /root/.cache/pip/wheels/dd/91/29/a79cecb328d01739e64017b6fb9a1ab9d8cb1853098ec5966d
Successfully built htmlmin


In [22]:
!pip install findspark pyspark pandas-profiling


In [24]:
!pip install sweetviz


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 41.3 MB/s eta 0:00:00


In [25]:
!pip install findspark pyspark sweetviz


In [32]:
import sweetviz as sv
# Convertir el DataFrame de cursos a pandas para el análisis
df_pandas = curso_df.toPandas()

# Generar el análisis de datos con Sweetviz
reporte = sv.analyze(df_pandas)
reporte.show_html('Reporte_Cursos.html', open_browser=False)

# Mostrar el reporte generado
reporte.show_html('Reporte_Cursos.html')

                                             |          | [  0%]   00:00 -> (? left)

Report Reporte_Cursos.html was generated.
Report Reporte_Cursos.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [33]:
reporte.show_html('data.html', open_browser=False)

Report data.html was generated.


#### Fase 2. Pre-procesamiento de datos

In [34]:
from pyspark.sql.functions import col

# Seleccionar y convertir columnas en el DataFrame de cursos
dataset = curso_df.select(
    col('id_curso'),
    col('id_estudiante').cast('int'),
    col('id_materia'),
    col('asistencia').cast('float'),
    col('nota_diagnostico').cast('float'),
    col('nota_parcial').cast('float'),
    col('nota_final').cast('float')
)

# Mostrar el DataFrame resultante
dataset.show()

+--------+-------------+----------+----------+----------------+------------+----------+
|id_curso|id_estudiante|id_materia|asistencia|nota_diagnostico|nota_parcial|nota_final|
+--------+-------------+----------+----------+----------------+------------+----------+
|     M3A|            2|       1MR|      90.0|             6.0|         6.5|      5.91|
|     M3A|            3|       1MR|      85.0|             7.0|         8.0|      8.79|
|     M3A|            5|       1MR|      95.0|             9.0|         9.5|      9.32|
|     M3A|            9|       1MR|      88.0|             8.0|         9.0|      9.13|
|     M3A|           10|       1MR|      92.0|             6.0|         6.5|      6.17|
|     M3A|            0|       1MR|     100.0|             9.5|         9.0|       9.4|
|     M3A|            1|       0MD|      75.0|             7.0|         8.5|      9.17|
|     M3A|            2|       0MD|      80.0|             8.0|         8.5|      8.53|
|     M3A|            3|       0

In [35]:
from pyspark.sql.functions import isnull, when, count, col

dataset.select([count(when(isnull(c), c)).alias(c) for c in dataset.columns]).show()

+--------+-------------+----------+----------+----------------+------------+----------+
|id_curso|id_estudiante|id_materia|asistencia|nota_diagnostico|nota_parcial|nota_final|
+--------+-------------+----------+----------+----------------+------------+----------+
|       0|            0|         0|         0|               0|           0|         0|
+--------+-------------+----------+----------+----------------+------------+----------+



In [36]:
# Drop missing values
dataset = dataset.replace('null', None)\
        .dropna(how='any')

In [37]:
dataset.show()

+--------+-------------+----------+----------+----------------+------------+----------+
|id_curso|id_estudiante|id_materia|asistencia|nota_diagnostico|nota_parcial|nota_final|
+--------+-------------+----------+----------+----------------+------------+----------+
|     M3A|            2|       1MR|      90.0|             6.0|         6.5|      5.91|
|     M3A|            3|       1MR|      85.0|             7.0|         8.0|      8.79|
|     M3A|            5|       1MR|      95.0|             9.0|         9.5|      9.32|
|     M3A|            9|       1MR|      88.0|             8.0|         9.0|      9.13|
|     M3A|           10|       1MR|      92.0|             6.0|         6.5|      6.17|
|     M3A|            0|       1MR|     100.0|             9.5|         9.0|       9.4|
|     M3A|            1|       0MD|      75.0|             7.0|         8.5|      9.17|
|     M3A|            2|       0MD|      80.0|             8.0|         8.5|      8.53|
|     M3A|            3|       0

In [60]:
from pyspark.ml.feature import StringIndexer

# Indexar 'id_curso' a 'id_curso_index'
indexer_curso = StringIndexer(
    inputCol='id_curso',
    outputCol='id_curso_index',
    handleInvalid='keep'
)

# Ajustar y transformar el DataFrame
dataset = indexer_curso.fit(dataset).transform(dataset)

# Verifica que la columna 'id_curso_index' fue creada
dataset.show()
print(dataset.columns)  # Asegúrate de que 'id_curso_index' está en las columnas

# Indexar 'id_materia' a 'id_materia_index'
indexer_materia = StringIndexer(
    inputCol='id_materia',
    outputCol='id_materia_index',
    handleInvalid='keep'
)

# Ajustar y transformar el DataFrame
dataset = indexer_materia.fit(dataset).transform(dataset)

# Verifica que la columna 'id_materia_index' fue creada
dataset.show()
print(dataset.columns)  # Asegúrate de que 'id_materia_index' está en las columna


+--------+-------------+----------+------------+----------+--------------+
|id_curso|id_estudiante|id_materia|nota_parcial|nota_final|id_curso_index|
+--------+-------------+----------+------------+----------+--------------+
|     M3A|            2|       1MR|         6.5|      5.91|           0.0|
|     M3A|            3|       1MR|         8.0|      8.79|           0.0|
|     M3A|            5|       1MR|         9.5|      9.32|           0.0|
|     M3A|            9|       1MR|         9.0|      9.13|           0.0|
|     M3A|           10|       1MR|         6.5|      6.17|           0.0|
|     M3A|            0|       1MR|         9.0|       9.4|           0.0|
|     M3A|            1|       0MD|         8.5|      9.17|           0.0|
|     M3A|            2|       0MD|         8.5|      8.53|           0.0|
|     M3A|            3|       0MD|         8.5|      8.63|           0.0|
|     M3A|            4|       0MD|         9.0|      9.24|           0.0|
|     M3A|            5| 

In [43]:
dataset.dtypes

[('id_curso', 'string'),
 ('id_estudiante', 'int'),
 ('id_materia', 'string'),
 ('asistencia', 'float'),
 ('nota_diagnostico', 'float'),
 ('nota_parcial', 'float'),
 ('nota_final', 'float')]

In [52]:
# Drop unnecessary columns
dataset = dataset.drop('asistencia')
dataset = dataset.drop('nota_diagnostico')
dataset.show()

+--------+-------------+----------+------------+----------+
|id_curso|id_estudiante|id_materia|nota_parcial|nota_final|
+--------+-------------+----------+------------+----------+
|     M3A|            2|       1MR|         6.5|      5.91|
|     M3A|            3|       1MR|         8.0|      8.79|
|     M3A|            5|       1MR|         9.5|      9.32|
|     M3A|            9|       1MR|         9.0|      9.13|
|     M3A|           10|       1MR|         6.5|      6.17|
|     M3A|            0|       1MR|         9.0|       9.4|
|     M3A|            1|       0MD|         8.5|      9.17|
|     M3A|            2|       0MD|         8.5|      8.53|
|     M3A|            3|       0MD|         8.5|      8.63|
|     M3A|            4|       0MD|         9.0|      9.24|
|     M3A|            5|       0MD|         9.0|      9.05|
|     M3A|            7|       0MD|         9.0|      9.71|
|     M3A|            8|       0MD|         8.5|      8.74|
|     M3A|            9|       0MD|     

In [64]:
from pyspark.ml.feature import VectorAssembler

# Lista de las características a usar en el VectorAssembler
required_features = ['id_curso_index',
                     'id_materia_index',
                     'nota_parcial',
                     'nota_final'
                    ]

# Verificar que todas las columnas necesarias están en el DataFrame
missing_features = [feature for feature in required_features if feature not in dataset.columns]
if missing_features:
    print(f"Las siguientes columnas están ausentes: {missing_features}")
else:
    # Crear el VectorAssembler
    assembler = VectorAssembler(inputCols=required_features, outputCol='features')

    # Transformar el DataFrame para incluir la columna de características
    transformed_data = assembler.transform(dataset)

    # Mostrar el DataFrame resultante con la columna 'features'
    transformed_data.show(truncate=False)

+--------+-------------+----------+------------+----------+--------------+----------------+-------------------------------+
|id_curso|id_estudiante|id_materia|nota_parcial|nota_final|id_curso_index|id_materia_index|features                       |
+--------+-------------+----------+------------+----------+--------------+----------------+-------------------------------+
|M3A     |2            |1MR       |6.5         |5.91      |0.0           |3.0             |[0.0,3.0,6.5,5.909999847412109]|
|M3A     |3            |1MR       |8.0         |8.79      |0.0           |3.0             |[0.0,3.0,8.0,8.789999961853027]|
|M3A     |5            |1MR       |9.5         |9.32      |0.0           |3.0             |[0.0,3.0,9.5,9.319999694824219]|
|M3A     |9            |1MR       |9.0         |9.13      |0.0           |3.0             |[0.0,3.0,9.0,9.130000114440918]|
|M3A     |10           |1MR       |6.5         |6.17      |0.0           |3.0             |[0.0,3.0,6.5,6.170000076293945]|
|M3A    

In [65]:
transformed_data.show()

+--------+-------------+----------+------------+----------+--------------+----------------+--------------------+
|id_curso|id_estudiante|id_materia|nota_parcial|nota_final|id_curso_index|id_materia_index|            features|
+--------+-------------+----------+------------+----------+--------------+----------------+--------------------+
|     M3A|            2|       1MR|         6.5|      5.91|           0.0|             3.0|[0.0,3.0,6.5,5.90...|
|     M3A|            3|       1MR|         8.0|      8.79|           0.0|             3.0|[0.0,3.0,8.0,8.78...|
|     M3A|            5|       1MR|         9.5|      9.32|           0.0|             3.0|[0.0,3.0,9.5,9.31...|
|     M3A|            9|       1MR|         9.0|      9.13|           0.0|             3.0|[0.0,3.0,9.0,9.13...|
|     M3A|           10|       1MR|         6.5|      6.17|           0.0|             3.0|[0.0,3.0,6.5,6.17...|
|     M3A|            0|       1MR|         9.0|       9.4|           0.0|             3.0|[0.0,

In [66]:
transformed_data.head()

Row(id_curso='M3A', id_estudiante=2, id_materia='1MR', nota_parcial=6.5, nota_final=5.909999847412109, id_curso_index=0.0, id_materia_index=3.0, features=DenseVector([0.0, 3.0, 6.5, 5.91]))

#### Fase 3. Entrenamiento del modelo


In [67]:
(training_data, test_data) = transformed_data.randomSplit([0.8,0.2])

In [77]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(labelCol='id_estudiante',
                            featuresCol='features',
                            maxDepth=5)

In [78]:
model = rf.fit(training_data)

In [70]:
# Predict with the test dataset
predictions = model.transform(test_data)

#### Fase 4. Evaluación del modelo

In [73]:
# Evaluate our model
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(
    labelCol='id_materia',
    predictionCol='prediction',
    metricName='accuracy'
)

In [74]:
# Accuracy
accuracy = evaluator.evaluate(predictions)
print('Test Accuracy =', accuracy)

Test Accuracy = 0.2857142857142857
